In [8]:
# 1. Importar librerias
from IPython import get_ipython #interfaz mejorada
from IPython.display import display #interfaz mejorada de los outputs
import tensorflow
import sys
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import numpy as np
import cv2
import os
import glob
from imutils import paths # enlista archivos de una ruta dada en las carpetas y subcarpetas
from tensorflow.keras.models import load_model
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from google.colab import drive
from tqdm import tqdm # muestra el  de avance cuando ejecutas un for in

In [9]:
# 2. Montar Google Drive, en esa ruta debe estar la carpeta model_1 con las sub carpetas de fotos esteril, mineral
drive.mount('/content/drive')
workdir_path = 'drive/MyDrive/Colab Notebooks/mineral_esteril'
os.chdir(workdir_path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


FileNotFoundError: [Errno 2] No such file or directory: 'drive/MyDrive/Colab Notebooks/mineral_esteril'

In [ ]:
# Define los hiperparámetros
INIT_LR = 1e-3 # tasa de aprendizaje inicial, controla la velocidad con la que el modelo ajusta sus pesos durante el entrenamiento
EPOCHS = 15 # número de épocas
BS = 30 # tamaño del lote, numero de ejemplos de entrenamiento en cada iteracion
dataset_path = 'model_1'


# Carga las imágenes y las etiquetas
print("[INFO] loading images...")
imagePaths = list(paths.list_images(dataset_path)) # lista de las imagenes con su ruta
data = [] # lista donde se almaceneran las imagenes para entregar
labels = [] # lista de la etiqueta de cada imagen

for imagePath in tqdm(imagePaths, desc="\tloading..."):
    label = imagePath.split(os.path.sep)[-2] # nombre de la carpeta de la imagen cargada, mineral o esteril
    image = cv2.imread(imagePath) # leer imagen
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # convertir a formato rgb de tensorflow
    image = cv2.resize(image, (224, 224))
    data.append(image) #se añade la imagen a la lista data
    labels.append(label)#se añade la etiqueta de la imagen a la lista data

print("labels: ", np.unique(labels)) # verificar que esteril este primero y luego mineral, ya que nuestra identificacion sera en orden 0 esteril 1 mineral


# Preprocesa los datos
print("\nlista data antes de la normalizacion:\n\n",data[2][110][50:55]) # dato rgb de los pixeles de las columnas 50 a 54 de la fila 110 de la tercera foto
data = np.array(data) / 255.0 # normalizar valores rgb de cada pixel de 0 a 1
print("\nlista data despues de la normalizacion:\n\n",data[2][110][50:55])# dato rgb estandarizado de los pixeles de las columnas 50 a 54 de la fila 110 de la tercera foto
labels = np.array(labels) # convierte la lista labels e un array de numpy
lb = LabelBinarizer() # funcion que binariza
labels = lb.fit_transform(labels) # transofmra categorias a binario
labels = to_categorical(labels) # one hot encoding


# Divide los datos en conjuntos de entrenamiento y prueba, se esta utilizando 80% para entregar y 20% para test
(trainX, testX, trainY, testY) = train_test_split(data, labels,
                                                  test_size=0.20, stratify=labels, random_state=42)


# Define el aumento de datos
trainAug = ImageDataGenerator(
    width_shift_range=0.1,# movimiendo horizontal aleatorio maximo en 10%
    height_shift_range=0.1,# movimiendo vertical aleatorio maximo en 10%
    zoom_range=0.1 #zoom aleatorio en 10%%
)


# Carga el modelo base VGG16
baseModel = VGG16(weights="imagenet", include_top=False,
                  input_tensor=Input(shape=(224, 224, 3)))


# Construye la cabeza del modelo
headModel = baseModel.output
headModel = MaxPooling2D(pool_size=(2, 2))(headModel)
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(128, activation="relu")(headModel)#128 neuronas
headModel = Dropout(0.2)(headModel)
headModel = Dense(2, activation="softmax")(headModel)


# Combina el modelo base y la cabeza
model = Model(inputs=baseModel.input, outputs=headModel)


# Congela las capas del modelo base
for layer in baseModel.layers:
    layer.trainable = False


# Compila el modelo
print("[INFO] compiling model...")
opt = Adam(learning_rate=INIT_LR, decay=INIT_LR / EPOCHS)
model.compile(loss="binary_crossentropy", optimizer=opt,
              metrics=["accuracy"])

[INFO] loading images...


	loading...: 100%|██████████| 10/10 [00:02<00:00,  4.39it/s]


labels:  ['esteril']
lista data antes de la normalizacion:
 [[ 88  91  98]
 [ 86  90  99]
 [ 98 101 110]
 [ 95  95 107]
 [ 98  95 103]
 [ 49  44  51]
 [ 75  75  73]
 [ 94  95 100]
 [ 89  89  94]
 [ 79  80  86]]
lista data despues de la normalizacion:
 [[0.34509804 0.35686275 0.38431373]
 [0.3372549  0.35294118 0.38823529]
 [0.38431373 0.39607843 0.43137255]
 [0.37254902 0.37254902 0.41960784]
 [0.38431373 0.37254902 0.40392157]
 [0.19215686 0.17254902 0.2       ]
 [0.29411765 0.29411765 0.28627451]
 [0.36862745 0.37254902 0.39215686]
 [0.34901961 0.34901961 0.36862745]
 [0.30980392 0.31372549 0.3372549 ]]
[INFO] compiling model...


/usr/local/lib/python3.11/dist-packages/keras/src/optimizers/base_optimizer.py:86: UserWarning: Argument `decay` is no longer supported and will be ignored.
  warnings.warn(


In [ ]:
# Entrena el modelo
print("[INFO] training head...")
H = model.fit(trainAug.flow(trainX, trainY, batch_size=BS),
        steps_per_epoch=len(trainX) // BS,
        validation_data=(testX, testY),
        validation_steps=len(testX) // BS,
        epochs=EPOCHS)

[INFO] training head...


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/15


/usr/local/lib/python3.11/dist-packages/keras/src/models/functional.py:237: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: ['keras_tensor']
Received: inputs=Tensor(shape=(None, 224, 224, 3))
  warnings.warn(msg)


20/20 ━━━━━━━━━━━━━━━━━━━━ 488s 25s/step - accuracy: 0.7267 - loss: 0.6076 - val_accuracy: 0.7815 - val_loss: 0.4371
Epoch 2/15
 1/20 ━━━━━━━━━━━━━━━━━━━━ 5:13 16s/step - accuracy: 0.8000 - loss: 0.3281

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/epoch_iterator.py:107: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


20/20 ━━━━━━━━━━━━━━━━━━━━ 100s 4s/step - accuracy: 0.8000 - loss: 0.3281 - val_accuracy: 0.7881 - val_loss: 0.4313
Epoch 3/15
20/20 ━━━━━━━━━━━━━━━━━━━━ 510s 24s/step - accuracy: 0.8324 - loss: 0.4001 - val_accuracy: 0.8212 - val_loss: 0.4197
Epoch 4/15
20/20 ━━━━━━━━━━━━━━━━━━━━ 158s 7s/step - accuracy: 0.8333 - loss: 0.4675 - val_accuracy: 0.8212 - val_loss: 0.4157
Epoch 5/15
20/20 ━━━━━━━━━━━━━━━━━━━━ 520s 24s/step - accuracy: 0.8021 - loss: 0.4052 - val_accuracy: 0.8344 - val_loss: 0.3838
Epoch 6/15
20/20 ━━━━━━━━━━━━━━━━━━━━ 158s 7s/step - accuracy: 0.8333 - loss: 0.4880 - val_accuracy: 0.8278 - val_loss: 0.3906
Epoch 7/15
20/20 ━━━━━━━━━━━━━━━━━━━━ 520s 24s/step - accuracy: 0.8656 - loss: 0.3429 - val_accuracy: 0.8212 - val_loss: 0.3843
Epoch 8/15
20/20 ━━━━━━━━━━━━━━━━━━━━ 102s 5s/step - accuracy: 0.7667 - loss: 0.4939 - val_accuracy: 0.8212 - val_loss: 0.3941
Epoch 9/15
20/20 ━━━━━━━━━━━━━━━━━━━━ 465s 24s/step - accuracy: 0.8659 - loss: 0.3440 - val_accuracy: 0.8212 - val_loss

In [ ]:
# Evalúa el modelo
print("[INFO] evaluating network...")
predIdxs = model.predict(testX, batch_size=BS)
predIdxs = np.argmax(predIdxs, axis=1)
print(classification_report(testY.argmax(axis=1), predIdxs,
                            target_names=lb.classes_))


# Calcula la matriz de confusión
cm = confusion_matrix(testY.argmax(axis=1), predIdxs)
total = sum(sum(cm))
acc = (cm[0, 0] + cm[1, 1]) / total
sensitivity = cm[0, 0] / (cm[0, 0] + cm[0, 1])
specificity = cm[1, 1] / (cm[1, 0] + cm[1, 1])
print(cm)
print("acc: {:.4f}".format(acc))


# Grafica la pérdida y la precisión del entrenamiento
N = EPOCHS
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), H.history["accuracy"], label="train_acc")
plt.plot(np.arange(0, N), H.history["val_accuracy"], label="val_acc")
plt.title("Training Loss and Accuracy on Testigos Yumpag")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
plt.show()


# Guarda el modelo
tensorflow.keras.models.save_model(model, "modelddh3.h5")

In [7]:
# Carga el modelo guardado
model_loaded = load_model("modelddh3.h5")


# Evalúa el modelo en el conjunto de prueba
dataset_path = ['test/esteriles', 'test/minerales']# carpeta y subcarpetas de las imagenes nuevas a testear
total_esteriles_predichos = 0
total_minerales_predichos = 0
total_esteriles_reales = 0
total_minerales_reales = 0

all_labels = [] # aqui se guardan los 0 y 1 de los labels orignales
all_predictions = [] # aqui las predicciones

for i in dataset_path: # se va recorrer la ruta, en este caso primero la subcarpeta esteriles de la carpeta test
    print("[INFO] testing images:", i)
    mimagePaths = list(paths.list_images(i)) #se enlista todas las imagenes de la carpeta
    mdata = [] #lista donde se almacenaran las imagenes formateadas para tensorflow
    mlabels = []#lista donde se almacenaran el codigo binarizado de la foto del testo, 0 esteril 1 mineral
    name_photo = []#lista del nombre de la foto

    for imagePath in mimagePaths: #formatear imagenes a tensorflor
        name = os.path.basename(imagePath)#nombre de la imagen
        mimage = cv2.imread(imagePath)#leer la imagen
        mimage = cv2.cvtColor(mimage, cv2.COLOR_BGR2RGB)#convertir el formato de la imagen a rgb
        mimage = cv2.resize(mimage, (224, 224)) #redimensionar la imagen a las mismas dimensiones de las fotos que se usaron para el entrenamiento
        mdata.append(mimage) # se añade la foto formateada a la lista mdata vacia de arriba
        name_photo.append(name)# se añade el nombre de la foto a la lista name_photo vacia de arriba
        label = 0 if i == "test/esteriles" else 1 # se indica 0 si el nombre de la carpeta de la imagen es esteril, sino es 1
        mlabels.append(label)# se añade el codigo 0 o 1 a la lista mlabels vacia de arriba

    mdata = np.array(mdata) / 255.0 # la lista de imagenes se tranforma a una lista de numpy y luego se estandariza de 0 a 1
    predIdxs = model_loaded.predict(mdata, batch_size=1) #prediccion de la lista mdata de imagenes formateadas procesando una imagen a la vez (batchsize)
    # la lista predIdxs tiene el siguiente formato:
    #  [[0.95, 0.05], Predicción para la imagen 1, aqui indica que 95% probabilidad es que sea esteril y 5% mineral
    #   [0.10, 0.90]]  Predicción para la imagen 2, aqui indica que 10% probabilidad es que sea esteril y 90% mineral
    print("\n\nlista predIdxs original en",i,":\n",predIdxs[:4])
    predIdxs = np.argmax(predIdxs, axis=1)#aqui seleccionas el indice de cada lista con el nro mayor de probabilidad, en una sola lista segun el ejemplo de arriba seria [0, 1]
    print("\nlista predIdxs con el indice de mayor probabilidad 0 esteril 1 mineral:\n",predIdxs[:4],"\n")
    all_labels.extend(mlabels)# se añaden la lista de los labels originales a la lista total 0 y 1
    all_predictions.extend(predIdxs)# se añaden la lista de las predicciones

    if i == "test/esteriles":#esto solo es para verificar el nro total de imagenes cargadas vs lo predicho
        # resta el total de fotos evaluadas menos la suma de predicciones
        # si todas las imagenes en esta parte fueran esteriles significa que la sumatoria de predIdx seria 0 y el resultado seria 100% de acierto
        esteriles_predichos = len(predIdxs) - np.sum(predIdxs)
        total_esteriles_predichos += esteriles_predichos
        total_esteriles_reales += len(mlabels)
    else:
        # como minerales tiene 1 de prediccion cuando es correcta, el numero de fotos acertadas es la suma de predIdxs
        minerales_predichos = np.sum(predIdxs)
        total_minerales_predichos += minerales_predichos
        total_minerales_reales += len(mlabels)


# Calcula las métricas de evaluación
precision_esteriles = precision_score(all_labels, all_predictions, pos_label=0)
precision_minerales = precision_score(all_labels, all_predictions, pos_label=1)
recall_esteriles = recall_score(all_labels, all_predictions, pos_label=0)
recall_minerales = recall_score(all_labels, all_predictions, pos_label=1)
f1_esteriles = f1_score(all_labels, all_predictions, pos_label=0)
f1_minerales = f1_score(all_labels, all_predictions, pos_label=1)

print(f"Precisión para esteriles: {precision_esteriles * 100:.2f}%")
print(f"Precisión para minerales: {precision_minerales * 100:.2f}%")
print(f"Recall para esteriles: {recall_esteriles * 100:.2f}%")
print(f"Recall para minerales: {recall_minerales * 100:.2f}%")
print(f"F1-score para esteriles: {f1_esteriles * 100:.2f}%")
print(f"F1-score para minerales: {f1_minerales * 100:.2f}%")

[INFO] testing images: test/esteriles
34/34 ━━━━━━━━━━━━━━━━━━━━ 20s 576ms/step


lista predIdxs original en test/esteriles :
 [[9.9999011e-01 9.8712226e-06]
 [9.9997652e-01 2.3432041e-05]
 [9.9965370e-01 3.4631338e-04]
 [9.9999726e-01 2.7144736e-06]]

lista predIdxs con el indice de mayor probabilidad 0 esteril 1 mineral:
 [0 0 0 0] 

[INFO] testing images: test/minerales
22/22 ━━━━━━━━━━━━━━━━━━━━ 13s 571ms/step


lista predIdxs original en test/minerales :
 [[1.5294567e-01 8.4705436e-01]
 [7.7250379e-01 2.2749622e-01]
 [9.9394339e-01 6.0566240e-03]
 [6.0236850e-04 9.9939764e-01]]

lista predIdxs con el indice de mayor probabilidad 0 esteril 1 mineral:
 [1 0 0 1] 

Precisión para esteriles: 80.95%
Precisión para minerales: 100.00%
Recall para esteriles: 100.00%
Recall para minerales: 63.64%
F1-score para esteriles: 89.47%
F1-score para minerales: 77.78%
